In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
import seaborn as sns
import requests


In [ ]:
# use this to keep the cells wider, very nice on wide screens, set the percentage as you like it
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
conf = {
    'font.size': 14.0,
    'axes.grid': True,
    'axes.axisbelow': True,
    'axes.edgecolor': 'black',
#    'axes.facecolor': '#E5E5EF',
#    'axes.facecolor': '#E0E0EF',
    'axes.labelcolor': 'black',
    'axes.titlesize': 15.6,
    'axes.labelsize': 'large',    
    'figure.figsize': (12, 7),
    'figure.titlesize': 'x-large',
    'grid.linewidth': 1.3,
    'xtick.labelsize': 'large',
    'ytick.labelsize': 'large',
    'xtick.color': 'black',
    'ytick.color': 'black',
    'legend.fontsize': 'large',
    'figure.figsize': (12., 8.)
}

plt.rcParams.update(conf)

### base loading and cleaning

In [ ]:
df_form = pd.read_excel('Feedback_Animefest_2019.xlsx', sheet_name='Form Responses 1')

In [ ]:
df_form['Timestamp'] = pd.to_datetime(df_form['Timestamp'])

In [ ]:
df_app = pd.read_excel('Feedback_Animefest_2019.xlsx', sheet_name='Appky')

In [ ]:
df_app['čas feedbacku'] = pd.to_datetime(df_app['čas feedbacku'])

In [ ]:
df_form = df_form.replace(to_replace={'😐v pohodě': 'v pohodě', '😃dobré': 'dobré', '😞špatné': 'špatné', '😐 v pohodě': 'v pohodě', '😃 dobré': 'dobré', '😞 špatné': 'špatné', '😃Ano': 'Ano', '😞Ne': 'Ne'})

In [ ]:
df_form.head()

##### several columns with same name, mutually exlusively filled, merging those

In [ ]:
duplicit_columns = [i for i in df_form.columns if i.endswith('.1')]
duplicit_columns

In [ ]:
[df_form[[i[:-2], i]].notnull().sum(axis=1).max() for i in duplicit_columns]

In [ ]:
for i in duplicit_columns:
    assert df_form[[i[:-2], i]].notnull().sum(axis=1).max() == 1  # only one column is filled, the other is N/A
    df_form.loc[df_form[i].notnull(), i[:-2]] = df_form[df_form[i].notnull()][i]
df_form = df_form.drop(columns=duplicit_columns)

In [ ]:
info_suffix = ' [Dozvěděl(a) jsem se zajímavé informace]'
fun_suffix = ' [Bavil(a) jsem se]'

##### renaming columns with additional space

In [ ]:
program_cols_to_strip_space = ['Aikido Ikigai Dojo Brno', 'Aikikai Aikido Brno', 'Kensei Dojo Brno']
replace_dict = {}
for col in program_cols_to_strip_space:
    replace_dict[f'{col} {info_suffix}'] = f'{col}{info_suffix}'
    replace_dict[f'{col} {fun_suffix}'] = f'{col}{fun_suffix}'
    replace_dict[f'Komentář: {col} '] = f'Komentář: {col}'
    
df_form = df_form.rename(columns=replace_dict)

In [ ]:
columns_series = df_form.columns.to_series()
program_columns_ratings = columns_series[columns_series.str.contains(info_suffix, regex=False) | columns_series.str.contains(fun_suffix, regex=False)]
program_columns_ratings.head()

In [ ]:
program_columns = pd.Series(program_columns_ratings.str.replace(info_suffix, '', regex=False).str.replace(fun_suffix, '', regex=False).unique())
program_columns.head()

##### some columns that should have same content have nans somewhere, fixing this

In [ ]:
weird_cols = ['2. světová válka z pohledu Japonska a život císaře Hirohita [Dozvěděl(a) jsem se zajímavé informace]', '2. světová válka z pohledu Japonska a život císaře Hirohita [Bavil(a) jsem se]']
df_form[df_form[weird_cols[0]] == 'Nedostal(a) jsem se'][weird_cols].head()

In [ ]:
# if one column marks not attended, and second is null, value is inferred
for i in program_columns:
    info_cond = df_form[i+info_suffix].isna() & df_form[i+fun_suffix].isin(['Nedostal(a) jsem se', 'Nezúčastnil(a) jsem se'])
    fun_cond = df_form[i+fun_suffix].isna() & df_form[i+info_suffix].isin(['Nedostal(a) jsem se', 'Nezúčastnil(a) jsem se'])
    df_form.loc[info_cond, i+info_suffix] = df_form[info_cond][i+fun_suffix]
    df_form.loc[fun_cond, i+fun_suffix] = df_form[fun_cond][i+info_suffix]


In [ ]:
df_form[df_form[weird_cols[0]] == 'Nedostal(a) jsem se'][weird_cols].head()

##### some columns that should have nan or rating have not attended, fixing this

In [ ]:
weird_cols = ['Maul Cosplay Q&A [Dozvěděl(a) jsem se zajímavé informace]', 'Maul Cosplay Q&A [Bavil(a) jsem se]']
df_form[df_form[weird_cols[1]] == 'v pohodě'][weird_cols].head()

In [ ]:
# if one column marks not attended, and second is rating, not attended is set to null
for i in program_columns:
    info_cond = df_form[i+fun_suffix].isin(['dobré', 'v pohodě', 'špatné']) & df_form[i+info_suffix].isin(['Nedostal(a) jsem se', 'Nezúčastnil(a) jsem se'])
    fun_cond = df_form[i+info_suffix].isin(['dobré', 'v pohodě', 'špatné']) & df_form[i+fun_suffix].isin(['Nedostal(a) jsem se', 'Nezúčastnil(a) jsem se'])
    df_form.loc[info_cond, i+info_suffix] = np.nan
    df_form.loc[fun_cond, i+fun_suffix] = np.nan


In [ ]:
df_form[df_form[weird_cols[1]] == 'v pohodě'][weird_cols].head()

##### some columns that should have same type of attendence have different. Converting 'Nedostal(a) jsem se' to 'Nezúčastnil(a) jsem se'

In [ ]:
weird_cols = ['12 ran cosplayerových [Dozvěděl(a) jsem se zajímavé informace]', '12 ran cosplayerových [Bavil(a) jsem se]']
df_form[df_form[weird_cols[0]] == 'Nedostal(a) jsem se'][weird_cols].tail()

In [ ]:
# if one column marks not attended, and second is null, value is inferred
for i in program_columns:
    info_cond = (df_form[i+info_suffix] == 'Nedostal(a) jsem se') & (df_form[i+fun_suffix] == 'Nezúčastnil(a) jsem se')
    fun_cond = (df_form[i+fun_suffix] == 'Nedostal(a) jsem se') & (df_form[i+info_suffix] == 'Nezúčastnil(a) jsem se')
    df_form.loc[info_cond, i+info_suffix] = df_form[info_cond][i+fun_suffix]
    df_form.loc[fun_cond, i+fun_suffix] = df_form[fun_cond][i+info_suffix]


In [ ]:
df_form[df_form[weird_cols[0]] == 'Nedostal(a) jsem se'][weird_cols].tail()

##### sanity checks

In [ ]:
for i in program_columns:
    info_sum = (df_form[i+info_suffix] == 'Nedostal(a) jsem se').sum()
    fun_sum = (df_form[i+fun_suffix] == 'Nedostal(a) jsem se').sum()
    both_sum = ((df_form[i+info_suffix] == 'Nedostal(a) jsem se') & (df_form[i+fun_suffix] == 'Nedostal(a) jsem se')).sum()
    if info_sum != fun_sum or fun_sum != both_sum:
        print(i, info_sum, fun_sum, both_sum)
    
    assert info_sum == fun_sum == both_sum

In [ ]:
for i in program_columns:
    info_sum = (df_form[i+info_suffix] == 'Nezúčastnil(a) jsem se').sum()
    fun_sum = (df_form[i+fun_suffix] == 'Nezúčastnil(a) jsem se').sum()
    both_sum = ((df_form[i+info_suffix] == 'Nezúčastnil(a) jsem se') & (df_form[i+fun_suffix] == 'Nezúčastnil(a) jsem se')).sum()
    if info_sum != fun_sum or fun_sum != both_sum:
        print(i, info_sum, fun_sum, both_sum)
    
    assert info_sum == fun_sum == both_sum

In [ ]:
for i in program_columns:
    info_sum = df_form[i+info_suffix].isin(['dobré', 'v pohodě', 'špatné', np.nan]).sum()
    fun_sum = df_form[i+fun_suffix].isin(['dobré', 'v pohodě', 'špatné', np.nan]).sum()
    both_sum = df_form[i+info_suffix].isin(['dobré', 'v pohodě', 'špatné', np.nan]).sum() & df_form[i+fun_suffix].isin(['dobré', 'v pohodě', 'špatné', np.nan]).sum()
    if info_sum != fun_sum or fun_sum != both_sum:
        print(i, info_sum, fun_sum, both_sum)
    
    assert info_sum == fun_sum == both_sum

In [ ]:
# replacing annoyingly long answer
df_form = df_form.replace({'chci hodnotit podrobně (je toho opravdu hodně, čím víc vyplníte, tím vděčnější budeme)': 'chci hodnotit podrobně'})

#### split of question between detailed and not detailed version

In [ ]:
# todo: dodělat
# přes varianty v Jak podrobně chcete hodnotit? a nany
df_form.columns[~df_form[df_form['Jak podrobně chcete hodnotit?'] == 'chci hodnotit stručně'].notnull().any()].to_series().tail(30).to_list()

In [ ]:
df_form.columns[~df_form[df_form['Jak podrobně chcete hodnotit?'] == 'chci hodnotit podrobně'].notnull().any()]

In [ ]:
# sloučit sloupce: 'Úvodní/závěrečné video', 'Úvodní a závěrečné video',

In [ ]:
columns_to_merge = {'Úvodní a závěrečné video': 'Úvodní/závěrečné video', 
                    'Připadá vám téma důležité pro zážitek na conu?': 'Připadá vám AF téma pro zážitek na conu důležité?', 
                    'Komentář: Připadá vám téma důležité pro zážitek na conu': 'Komentář: Připadá vám téma pro zážitek na conu důležité?',
                    'Komentář: Letošní téma a frakce:': 'Komentář: Letošní téma a frakce',
                    'Co se vám na aplikaci líbilo, v čem vám pomohla?': 'Co se vám na aplikaci líbilo? Co chcete do příště změnit nebo doplnit?'
                   }
for col_from, col_to in columns_to_merge.items():
    assert df_form[[col_from, col_to]].notnull().sum(axis=1).max() == 1  # only one column is filled, the other is N/A
    df_form.loc[df_form[col_from].notnull(), col_to] = df_form[df_form[col_from].notnull()][col_from]
df_form = df_form.drop(columns=columns_to_merge.keys())

In [ ]:
df_form.columns[~df_form[df_form['Jak podrobně chcete hodnotit?'] == 'chci hodnotit podrobně'].notnull().any()]

##### cleaning open questions by replacing nan by empty string

In [ ]:
open_answers = [i for i in df_form.columns if len(df_form[i].unique()) > 20 or 'Komentář:' in i] + ['Poznámky k doprovodnému programu']
len(open_answers)

In [ ]:
for col in open_answers:
    df_form[col] = df_form[col].fillna('')

### starting analyzing apps

In [ ]:
df_app.head()

In [ ]:
df_app.describe()

In [ ]:
df_app['uživatel'].value_counts().head()

In [ ]:
(df_app['uživatel'] + df_app['program ID'].astype(str)).value_counts().hist()

In [ ]:
df_app['uživatel'].value_counts().head()

In [ ]:
(df_app['uživatel'] + df_app['program ID'].astype(str)).value_counts().value_counts()

#### cleaning data, keeping only last rating

In [ ]:
df_app['user_program'] = df_app['uživatel'] + ' ' + df_app['program ID'].astype(str)

In [ ]:
df_app[df_app['user_program'].isin(df_app['user_program'].value_counts()[df_app['user_program'].value_counts() > 1].index)].sort_values(by='název').head()

In [ ]:
df_app.head()

In [ ]:
df_app = df_app.groupby(['user_program']).apply(lambda x: x.sort_values(by='čas feedbacku', ascending=False).head(1))

In [ ]:
df_app.sort_values(by='název').head()

#### merging together both app and form

In [ ]:
df_form.head()

In [ ]:
df_app.head()

In [ ]:
program_ratings.head()

In [ ]:
app_fun_ratings = pd.concat((df_app.groupby('název').apply(lambda x: (x['fun'] == 1).sum()), df_app.groupby('název').apply(lambda x: (x['fun'] == 2).sum()), df_app.groupby('název').apply(lambda x: (x['fun'] == 3).sum())), axis=1)
app_fun_ratings.index += fun_suffix
app_fun_ratings = app_fun_ratings.rename(columns={0: 'špatné', 1: 'v pohodě', 2: 'dobré'})
app_fun_ratings = app_fun_ratings.T
app_fun_ratings.head()

In [ ]:
app_info_ratings = pd.concat((df_app.groupby('název').apply(lambda x: (x['informace'] == 1).sum()), df_app.groupby('název').apply(lambda x: (x['informace'] == 2).sum()), df_app.groupby('název').apply(lambda x: (x['informace'] == 3).sum())), axis=1)
app_info_ratings.index += info_suffix
app_info_ratings = app_info_ratings.rename(columns={0: 'špatné', 1: 'v pohodě', 2: 'dobré'})
app_info_ratings = app_info_ratings.T
app_info_ratings.head()

In [ ]:
app_ratings = pd.concat((app_fun_ratings, app_info_ratings), axis=1)
app_ratings = app_ratings.reindex(sorted(app_ratings.columns), axis=1)
app_ratings

### base EDA

In [ ]:
df_form.head()

In [ ]:
df_form.describe()

In [ ]:
[(i, len(df_form[i].unique())) for i in df_form.columns if len(df_form[i].unique()) < 10]

In [ ]:
df_form.columns.map(lambda i: len(df_form[i].unique())).to_series().value_counts().sort_index().plot.bar(figsize=(25, 10))

In [ ]:
df_form.columns.map(lambda i: sum(df_form[i].notnull())).to_series().value_counts().sort_index().plot.bar(figsize=(25, 10))

In [ ]:
attend_columns = [i for i in df_form.columns if 'Nezúčastnil(a) jsem se' in df_form[i].values]

In [ ]:
attend_columns[:10]

In [ ]:
len(attend_columns)

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) <= 8])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) < 10])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) < 20])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) <= 8 and sum(df_form[i].notnull()) > 100])

In [ ]:
all_attends = df_form[attend_columns].stack()
all_attends[~all_attends.isna()].value_counts().plot.pie()

In [ ]:
all_attends[~all_attends.isna()].value_counts().plot.barh()

In [ ]:
all_attends[(~all_attends.isna()) & ('Nezúčastnil(a) jsem se' != all_attends)].value_counts().plot.barh()

In [ ]:
df_app.head()

In [ ]:
df_app.describe()

In [ ]:
def abs_vals(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return '{:d}'.format(absolute)


In [ ]:
feedback_geo = df_form['Bydliště'].value_counts()
feedback_geo.plot.pie(autopct=partial(abs_vals, allvals=feedback_geo))

In [ ]:
#for i in [i for i in df_form.columns if len(df_form[i].unique()) <= 8 and sum(df_form[i].notnull()) > 100]:
#    plt.figure()
#    feedback_i = df_form[i].value_counts()
#    feedback_i.plot.pie(autopct=partial(abs_vals, allvals=feedback_i))
#    plt.show()
    
# todo: dodat filtrování, že tam kde je pod 20 co se zúčastnilia ty vyhodit, udělat i analýzu toho, kdo jak vyplnil zúčastnil/nezúčastnil jsem se

In [ ]:
[i for i in df_form.columns if len(df_form[i].unique()) <= 8 and '[Dozvěděl(a) jsem se zajímavé informace]' not in i and '[Bavil(a) jsem se]' not in i and 'Komentář:' not in i]

In [ ]:
cols_to_plot = [i for i in df_form.columns if len(df_form[i].unique()) <= 8 and '[Dozvěděl(a) jsem se zajímavé informace]' not in i and '[Bavil(a) jsem se]' not in i and 'Komentář:' not in i]
cols = 3
fig, axes = plt.subplots(int(np.ceil(len(cols_to_plot) / cols)), cols, figsize=(35, 120))

for i, col in enumerate(cols_to_plot):
    ax = axes[i // cols, i % cols]
    #plt.figure()
    feedback_i = df_form[col].value_counts()
    feedback_i.plot.pie(autopct=partial(abs_vals, allvals=feedback_i), ax=ax, y=None)
plt.show()
    
# todo: dodat filtrování, že tam kde je pod 20 co se zúčastnilia ty vyhodit, udělat i analýzu toho, kdo jak vyplnil zúčastnil/nezúčastnil jsem se

In [ ]:
feedback_times = df_form['Timestamp'].value_counts(sort=False).copy().reindex().resample('1D', how='count')
feedback_times.plot.bar()
plt.title('Počet vyplnění za den, vše červen')
#plt.gca().set_xticklabels([dt.strftime("%#d. %#m.") for dt in feedback_times.index])
plt.gca().set_xticklabels([dt.strftime("%#d.") for dt in feedback_times.index])
plt.xticks(rotation=0)
plt.show()

In [ ]:
program_ratings = df_form[program_columns_ratings].apply(lambda x: x.value_counts(), axis=0)

In [ ]:
attended_col = (program_ratings.loc['dobré'].fillna(0) + program_ratings.loc['v pohodě'].fillna(0) + program_ratings.loc['špatné'].fillna(0)).rename('Zúčastnil(a) jsem se')
program_attends = pd.concat((attended_col, program_ratings.loc['Nedostal(a) jsem se']), axis=1)
program_attends.head()

In [ ]:
program_ratings = program_ratings.drop('Nezúčastnil(a) jsem se')

In [ ]:
program_ratings

In [ ]:
def remove_suffix(string):
    return string.replace(' [Dozvěděl(a) jsem se zajímavé informace]', '').replace(' [Bavil(a) jsem se]', '')
program_ratings.groupby(program_ratings.columns.map(remove_suffix), axis=1).max()

In [ ]:
type_to_color = {'přednášky': 'g', 'workshopy': 'purple', 'soutěže': 'yellow', 'doplňkový program': 'b', 'promítání': 'r', 'divadlo': 'pink'}
def patch_label_coords(p):
    offset = 0
    if p.get_width() < 0:
        offset = - 0.03
    return p.get_x() + p.get_width() + offset, p.get_y() - 0.25

In [ ]:
program_attends.T.groupby(program_ratings.columns.map(remove_suffix), axis=1).max().T.sort_values('Zúčastnil(a) jsem se', ascending=True).plot.barh(stacked=True, figsize=(30, 50))

#### splitting program to categories

In [ ]:
categories = {
    'soutěže': [ 
        'Animekvíz', 'Cosplay debut', 'Cosplay soutěž', 'Cosplay video', 'Festdance', 'Soutěžní AMV', 'Vyhlášení výsledků', 
    ], 
    'přednášky': [
        '12 ran cosplayerových', '2. světová válka z pohledu Japonska a život císaře Hirohita', '3D tisk v cosplayi: Od modelování po barvení', 'Alternativní móda: Od ponožky po klobouk', 'An Introduction to Leather Crafting', 
        'Anime světy, ve kterých (ne)chcete žít', 'BJD od Ludvíka XIV. až po BTS', 'Bojuj podle svého charakteru', 'Cosplay and Otaku History in Japan', 'Cosplay armor: From reference to wearable costume', 
        'Crossdressing - the hell is dat?', 'Designing Male Characters Fanservice: The Struggle', 'Do Japonska za idoly', 'Doll Photography', 'Fake Is Sad/Bootleg Panel', 'Filmoví skladatelé Japonska č. 2', 
        'Gača hry aneb Lootboxy na asijský způsob', 'Gejša: Tajemství za stěnami čajovny', 'Ghibli známé neznámé', 'Godzilla, just a rubber monster, right?', 'Gothic – hudba, móda, subkultura', 
        'Historická přesnost zbrojí v anime na vybraných příkladech', 'Indonésie – země ohně', 'Jak na anime make-up', 'Jak na psaní – Od teorie k praxi', 'Jak se připravit na cosplay focení', 'Jak sehnat (nejen) jaoi v totalitním státu', 
        'Jak si plnit sny pomocí anime', 'Japanacorps, WTF?!', 'Japonský rok v emodži', 'Letem jiným světem', 'Lolitou každý den', 'Líčení pro trdla od trdla', 'Madoka Magica a ti druzí, co selhali', 'Manga novinky u nás!', 
        'Manga pro dospívající dívky: Co na ní milujeme a nesnášíme', 'Maul Cosplay Q&A', 'My Hero Academy', 'Módní přehlídka 101', 'Nebojte se zahraničních soutěží!', 'Nový kreslíř na scéně: umělá inteligence!', 
        'Očekávání vs. realita: Člověk 2.0, mecha a vesmírné lodě', 'Pen & Paper role-playing hry', 'Proč (ne)jet do Jižní Koreje', 'Receptář tetičky Mitsu nejen pro začínající lolity', 'Remaky anime', 'Reputace & renesance D&D', 
        'Resident Evil – Od zrození po současnost', 'Rámen po česku', 'Sekiro, Nioh a ti další…', 'Slasti a strasti japonského randění', 'Slavnostní zahájení + křest Vějíře', 'Stereotyp – NÁMITKA!', 'Svět japonských idolů', 
        'Taková normální japonská rodinka', 'Viktoriánské úpravy vlasů', 'World cosplay panel', 'Zakončení conu', 'Zákeřné vody videoherního pirátství', 
    ], 
    'workshopy': [
        'Aikido Ikigai Dojo Brno', 'Aikikai Aikido Brno', 'Boj dýkou a nožem', 'Boj tesákem', 'Choker', 'Cosplay Act Workshop', 'Yoshinkan aikido', 'Kaligrafie', 'Kendo', 'Kensei Dojo Brno', 'Kimono workshop', 'Sraz AMV', 
        'Háčkování plyšových zvířátek', 'I pád je posun vpřed', 'Jak se při cosplayi neztrapnit se zbraní', 'Jo, bokken, tanto aneb Obrana proti japonským zbraním', 'Květiny Ikebana', 'Mák a jeho klacek aneb I monk si s holí leccos dovolí', 
        'Náušnice ze skleněných kapek', 'Od základů po pokročilejší retuše nejen cosplay fotografie', 'Omalovánky', 'Origami Workshop', 'Rozcvička pro družinku aneb Dračí doupě po ránu', 'Rytíř, meč a štít aneb Líný šerm', 
        'Růže ze saténových stužek', 'Sebeobrana nejen pro dámy', 'Sebeobrana po japonsku', 'Taneční k-pop workshop', 'Vyšívání', 'Válečník mazlivě obouručákem tě mlaskne', 'Výroba vlastního diáře/skicáku', 
        'Workshop animace v Clip Studio Paint', 'Základy svícení s profesionálními záblesky a jejich modifikátory'
    ], 
    'promítání': [
        '3D holka', 'A to je špatně, když se chodím seznamovat do kobek? Orionův šíp', 'Bůh žehnej tomuto nádhernému světu', 'Fashion Mix', 'Gamers!', 'Já malý čarodějka', 'Květina zaslíbená rannímu loučení', 'Lolita Music Mix: Novinky', 
        'Mirai, dívka z budoucnosti', 'Módní kolekce 2019', 'Noc je krátká, tak kráčej dál, děvče', 'Penguin Highway', 'Pohodářky', 'ReLIFE', 'Symfonie z jiného světa', 'Tipy pro život mimo realitu', 'Vzestup hrdiny štítu', 
        'Zombie Land Saga', 
    ],
    'doplňkový program': [
        'AMV Budíček', 'AMV Mortal Combat', 'AMV Nočník', 'AMV Večerníček', 'BTS Army sraz', 'Vějíř – autogramiáda a beseda', 'Anime novinky očima protřelých fanoušků', 'Budoucnost českých anime conů', 
        'K-pop – random dance CZHW', 'Sčítání žije!', 'Turnaj v šermu Aréna Alerie', 'Dračí doupě', 
    ],
    'divadlo': ['Festovní koncert', 'Kamui – The Samurai Sword Artists', 'Lolita Fashion Show', 'Malé divadlo kjógenu', ],
    }

In [ ]:
col_to_category = {col: category for category, cols in categories.items() for col in cols}
col_to_category

In [ ]:
program_attendance = program_attends.T.groupby(program_ratings.columns.map(remove_suffix), axis=1).max().T['Zúčastnil(a) jsem se'].to_frame()
program_attendance['type'] = program_attendance.index.to_series().map(col_to_category)

sorted_attendance = program_attendance.sort_values(by='Zúčastnil(a) jsem se', ascending=True)
sorted_attendance['Zúčastnil(a) jsem se'].plot.barh(stacked=True, figsize=(30, 50), color=sorted_attendance['type'].map(type_to_color))
for p in plt.gca().patches:
    plt.gca().annotate("%.0f" % p.get_width(), patch_label_coords(p), xytext=(5, 10), textcoords='offset points')

In [ ]:
program_attendance['type'] = program_attendance.index.to_series().map(col_to_category)
program_attendance.head()

In [ ]:
program_attendance.groupby('type')['Zúčastnil(a) jsem se'].sum().sort_values(ascending=True).plot.barh()

In [ ]:
program_attendance.groupby('type')['Zúčastnil(a) jsem se'].mean().sort_values(ascending=True).plot.barh()

In [ ]:
program_attendance.groupby('type')['Zúčastnil(a) jsem se'].count().sort_values(ascending=True).plot.barh()

In [ ]:
sns.violinplot(x='type', y='Zúčastnil(a) jsem se', data=program_attendance, inner='stick', bw=.3)
plt.title('návštěvnost')
plt.xticks(rotation=15)

In [ ]:
rating_to_num = {'špatné': -1, 'v pohodě': 0, 'dobré': 1}
fill_to_num = {'chci hodnotit stručně': -1, 'chci hodnotit podrobně': 1}
app_to_num = {'Ne': -1, 'Ne, nevědel sem o ní': -1, 'Ne, z jiného důvodu': -1, 'Ne, nemám podporované zařízení': -1, 'Ne, nedokázal jsem ji spustit': -1, 'Ne, nezajímá mě': -1, np.nan: 0, 'Ano': 1}
bool_to_num = {'Ne': -1, np.nan: 0, 'Ano': 1}
age_to_num = {'13-15': 1, '16-18': 2, '19-25': 3, '26 a více': 4}
    
#sns.pairplot(df_form[cols_to_plot], hue='Pohlaví')
plt.figure(figsize=(20, 20))
g = sns.pairplot(df_form[['Věk', 'Pohlaví', 'Jak podrobně chcete hodnotit?', 'Připadá vám AF téma pro zážitek na conu důležité?', 'Používali jste Animefest aplikaci?']].replace({
    'Věk': age_to_num, 'Jak podrobně chcete hodnotit?': fill_to_num, 'Připadá vám AF téma pro zážitek na conu důležité?': bool_to_num, 'Používali jste Animefest aplikaci?': app_to_num
}), hue='Pohlaví')
[plt.setp(ax.xaxis.get_label(), rotation=15) for ax in g.axes.flat]
[plt.setp(ax.yaxis.get_label(), rotation=75) for ax in g.axes.flat]


In [ ]:
program_ratings_fun = program_ratings[program_columns + fun_suffix].loc[['dobré', 'špatné', 'v pohodě']].T.fillna(0)
program_ratings_fun.head()

In [ ]:
program_ratings_info = program_ratings[program_columns + info_suffix].loc[['dobré', 'špatné', 'v pohodě']].T.fillna(0)
program_ratings_info.head()

In [ ]:
program_fun_score = (program_ratings_fun['dobré'] * 1 + program_ratings_fun['v pohodě'] * 0 + program_ratings_fun['špatné'] * -1) / program_ratings_fun.sum(axis=1)
program_fun_score.index = program_fun_score.index.str.replace(fun_suffix, '', regex=False)
program_fun_score.head()

In [ ]:
program_info_score = (program_ratings_info['dobré'] * 1 + program_ratings_info['v pohodě'] * 0 + program_ratings_info['špatné'] * -1) / program_ratings_info.sum(axis=1)
program_info_score.index = program_info_score.index.str.replace(info_suffix, '', regex=False)
program_info_score.head()


In [ ]:
program_fun_score = program_fun_score.to_frame('score')
program_fun_score['type'] = program_fun_score.index.to_series().map(col_to_category)
program_fun_score.head()

In [ ]:
program_info_score = program_info_score.to_frame('score')
program_info_score['type'] = program_info_score.index.to_series().map(col_to_category)
program_info_score.head()

In [ ]:
sorted_score = program_fun_score.sort_values(by='score', ascending=True)
sorted_score['score'].plot.barh(figsize=(30, 50), color=sorted_score['type'].map(type_to_color))
for p in plt.gca().patches:
    plt.gca().annotate("%.2f" % p.get_width(), patch_label_coords(p), xytext=(5, 10), textcoords='offset points')

In [ ]:
sorted_score = program_info_score.sort_values(by='score', ascending=True)
sorted_score['score'].plot.barh(figsize=(30, 50), color=sorted_score['type'].map(type_to_color))
for p in plt.gca().patches:
    plt.gca().annotate("%.2f" % p.get_width(), patch_label_coords(p), xytext=(5, 10), textcoords='offset points')

In [ ]:
sns.violinplot(x='type', y='score', data=program_fun_score, inner='stick', bw=.3)
plt.title('zábavnost')
plt.xticks(rotation=15)

In [ ]:
sns.violinplot(x='type', y='score', data=program_info_score, inner='stick', bw=.3)
plt.title('informovanost')
plt.xticks(rotation=15)

In [ ]:
### can't see number of them, need parallel sets

In [ ]:
{i: df_form[i].unique() for i in cols_to_plot}

In [ ]:
import plotly.graph_objects as go

titanic_df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/titanic.csv")

# Create dimensions
class_dim = go.parcats.Dimension(
    values=titanic_df.Pclass,
    categoryorder='category ascending', label="Class"
)

gender_dim = go.parcats.Dimension(values=titanic_df.Sex, label="Gender")

survival_dim = go.parcats.Dimension(
    values=titanic_df.Survived, label="Outcome", categoryarray=[0, 1], 
    ticktext=['perished', 'survived']
)

# Create parcats trace
color = titanic_df.Survived;
colorscale = [[0, 'lightsteelblue'], [1, 'mediumseagreen']];

fig = go.Figure(data = [go.Parcats(dimensions=[class_dim, gender_dim, survival_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [ ]:
titanic_df.head()

In [ ]:

df_form['af_aplikace_bool'] = df_form['Používali jste Animefest aplikaci?'].replace({'Ne, nedokázal jsem ji spustit': 'Ne', 'Ne, nemám podporované zařízení': 'Ne', 'Ne, nevědel sem o ní': 'Ne', 'Ne, nezajímá mě': 'Ne', 'Ne, z jiného důvodu': 'Ne'})

# Create dimensions
age_dim = go.parcats.Dimension(
    values=df_form['Věk'],
    categoryorder='category ascending', label='Věk'
)

gender_dim = go.parcats.Dimension(
    values=df_form['Pohlaví'],
    categoryorder='category ascending', label='Pohlaví'
)

place_dim = go.parcats.Dimension(
    values=df_form['Bydliště'],
    categoryorder='category ascending', label='Bydliště'
)

detail_dim = go.parcats.Dimension(
    values=df_form['Jak podrobně chcete hodnotit?'],
    categoryorder='category ascending', label='Podrobné hodnocení'
)

app_detailed_dim = go.parcats.Dimension(
    values=df_form['Používali jste Animefest aplikaci?'],
    categoryorder='category ascending', label='Používání AF aplikace'
)

app_dim = go.parcats.Dimension(
    values=df_form['Používali jste Animefest aplikaci?'].replace({'Ne, nedokázal jsem ji spustit': 'Ne', 'Ne, nemám podporované zařízení': 'Ne', 'Ne, nevědel sem o ní': 'Ne', 'Ne, nezajímá mě': 'Ne', 'Ne, z jiného důvodu': 'Ne'}),
    categoryorder='category ascending', label='Používání AF aplikace'
)


souvenirs_price_dim = go.parcats.Dimension(
    values=df_form['Cena AF suvenýrů je přiměřená?'],
    categoryorder='category ascending', label='Cena suvenýrů'
)

more_souvenirs_dim = go.parcats.Dimension(
    values=df_form['Uvítali byste nějaké další suvenýry?'],
    categoryorder='category ascending', label='Více suvenýrů?'
)

# not filled in detailed version
factions_dim = go.parcats.Dimension(
    values=df_form['Letošní téma a frakce'],
    categoryorder='category ascending', label='téma a frakce'
)

video_dim = go.parcats.Dimension(
    values=df_form['Úvodní/závěrečné video'],
    categoryorder='category ascending', label='Úvodní/závěrečné video'
)

points_dim = go.parcats.Dimension(
    values=df_form['Sbírání bodů'],
    categoryorder='category ascending', label='Sbírání bodů'
)

theme_dim = go.parcats.Dimension(
    values=df_form['Připadá vám AF téma pro zážitek na conu důležité?'],
    categoryorder='category ascending', label='Je AF téma důležité?'
)


# Create parcats trace
#color = df_form['Pohlaví'].replace({'muž': 0, 'žena': 0.3, 'jiné': 0.6, np.nan: 1})
color = df_form['Úvodní/závěrečné video'].replace({'dobré': 0, 'v pohodě': 0.2, 'špatné': 0.4, 'Nevím o co jde': 0.6, np.nan: 1})
#colorscale = [[0, 'steelblue'], [0.3, 'indianred'], [0.6, 'gray'], [1, 'dimgray']];

fig = go.Figure(data = [go.Parcats(dimensions=
#                                  [age_dim, gender_dim, app_dim, souvenirs_price_dim, video_dim, theme_dim],
                                   [age_dim, gender_dim, video_dim, theme_dim],
#        line={'color': color, 'colorscale': colorscale},
        line={'color': color},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [ ]:
df = df_form[['Věk', 'Pohlaví', 'Kromě letošního ročníku jsem na Animefestu byl(a)', 'Připadá vám AF téma pro zážitek na conu důležité?', 'Úvodní/závěrečné video']].dropna()
df['Kromě letošního ročníku jsem na Animefestu byl(a)'] = df['Kromě letošního ročníku jsem na Animefestu byl(a)'].replace({'6x': '6x+', '7x': '6x+', '8x': '6x+', '9x': '6x+', 
                                                                                                                           '10x': '6x+', '11x': '6x+', '12x': '6x+', '13x': '6x+', '14x': '6x+'})
# Create dimensions
age_dim = go.parcats.Dimension(
    values=df['Věk'],
    categoryorder='category ascending', label='Věk'
)

gender_dim = go.parcats.Dimension(
    values=df['Pohlaví'],
    categoryorder='category ascending', label='Pohlaví'
)

num_visits_dim = go.parcats.Dimension(
    values=df['Kromě letošního ročníku jsem na Animefestu byl(a)'],
    categoryorder='category ascending', label='Kolikrát předtím na AF'
)

video_dim = go.parcats.Dimension(
    values=df['Úvodní/závěrečné video'],
    categoryorder='category ascending', label='Úvodní/závěrečné video'
)

theme_dim = go.parcats.Dimension(
    values=df['Připadá vám AF téma pro zážitek na conu důležité?'],
    categoryorder='category ascending', label='Je AF téma důležité?'
)


# Create parcats trace
#color = df_form['Pohlaví'].replace({'muž': 0, 'žena': 0.3, 'jiné': 0.6, np.nan: 1})
color = df['Úvodní/závěrečné video'].replace({'dobré': 0, 'v pohodě': 0.2, 'špatné': 0.4, 'Nevím o co jde': 0.6, np.nan: 1})
#colorscale = [[0, 'steelblue'], [0.3, 'indianred'], [0.6, 'gray'], [1, 'dimgray']];

fig = go.Figure(data = [go.Parcats(dimensions=
                                   [age_dim, gender_dim, num_visits_dim, video_dim, theme_dim],
#        line={'color': color, 'colorscale': colorscale},
        line={'color': color},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [ ]:
df = df_form[['Věk', 'Pohlaví', 'Cena AF suvenýrů je přiměřená?']].dropna()

# Create dimensions
age_dim = go.parcats.Dimension(
    values=df['Věk'],
    categoryorder='category ascending', label='Věk'
)

gender_dim = go.parcats.Dimension(
    values=df['Pohlaví'],
    categoryorder='category ascending', label='Pohlaví'
)

souvenirs_price_dim = go.parcats.Dimension(
    values=df['Cena AF suvenýrů je přiměřená?'],
    categoryorder='category ascending', label='Cena suvenýrů'
)

# Create parcats trace
color = df_form['Pohlaví'].replace({'muž': 0, 'žena': 0.3, 'jiné': 0.6, np.nan: 1})
colorscale = [[0, 'steelblue'], [0.3, 'indianred'], [0.6, 'gray'], [1, 'dimgray']];

fig = go.Figure(data = [go.Parcats(dimensions=[age_dim, gender_dim, souvenirs_price_dim],
        #line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

In [ ]:
# for souvenirs price

df = df_form[['Věk', 'Bydliště', 'Pohlaví', 'Cena vstupného je…', 'Cena AF suvenýrů je přiměřená?']].dropna()

# Create dimensions
age_dim = go.parcats.Dimension(
    values=df['Věk'],
    categoryorder='category ascending', label='Věk'
)

place_dim = go.parcats.Dimension(
    values=df['Bydliště'],
    categoryorder='category ascending', label='Bydliště'
)

gender_dim = go.parcats.Dimension(
    values=df['Pohlaví'],
    categoryorder='category ascending', label='Pohlaví'
)

ticket_price_dim = go.parcats.Dimension(
    values=df['Cena vstupného je…'],
    categoryorder='category ascending', label='Cena vstupenek'
)

souvenirs_price_dim = go.parcats.Dimension(
    values=df['Cena AF suvenýrů je přiměřená?'],
    categoryorder='category ascending', label='Cena suvenýrů'
)

# Create parcats trace
color = df['Pohlaví'].replace({'muž': 0, 'žena': 0.3, 'jiné': 0.6, np.nan: 1})
colorscale = [[0, 'steelblue'], [0.3, 'indianred'], [0.6, 'gray'], [1, 'dimgray']];

fig = go.Figure(data = [go.Parcats(dimensions=[age_dim, gender_dim, ticket_price_dim, souvenirs_price_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()

'Dám i víc' dali jen muži 19+

### counting conditional probabilities

In [ ]:
def cond_probs(df, factors, column):
    temp_df = df.groupby(factors)[column].value_counts() / df.groupby(factors)[column].count()
    temp_df = temp_df.unstack(level=-1)
    if np.nan in temp_df.columns:
        temp_df = temp_df.drop(columns=np.nan)
    return temp_df

In [ ]:
cond_probs(df_form, 'Pohlaví', 'Cena AF suvenýrů je přiměřená?')

In [ ]:
cond_probs(df_form[df_form['Pohlaví'] != 'jiné'], ['Věk', 'Pohlaví'], 'Cena AF suvenýrů je přiměřená?')

In [ ]:
cond_probs(df_form[df_form['Pohlaví'] != 'jiné'], ['Věk', 'Pohlaví'], 'Používali jste Animefest aplikaci?')

In [ ]:
cond_probs(df_form[df_form['Pohlaví'] != 'jiné'], ['Věk', 'Pohlaví'], 'Úvodní/závěrečné video')

In [ ]:
cond_probs(df_form[df_form['Pohlaví'] != 'jiné'], ['Věk', 'Pohlaví'], 'Připadá vám AF téma pro zážitek na conu důležité?')

In [ ]:
cond_probs(df_form, ['Připadá vám AF téma pro zážitek na conu důležité?'], 'Úvodní/závěrečné video')

In [ ]:
cond_probs(df_form[df_form['Pohlaví'] != 'jiné'], ['Věk', 'Pohlaví', 'Úvodní/závěrečné video'], 'Připadá vám AF téma pro zážitek na conu důležité?')

In [ ]:
cols_to_plot

In [ ]:
df_form.head()

In [ ]:
titanic_df.head()

In [ ]:
people_ratings = df_form[program_columns_ratings].apply(lambda x: x.value_counts(), axis=1)
people_ratings = people_ratings.dropna(axis='rows', how='all', subset=['dobré', 'v pohodě', 'špatné']).fillna(0)
people_ratings.head()

In [ ]:
people_score = (people_ratings['dobré'] * 1 + people_ratings['v pohodě'] * 0 + people_ratings['špatné'] * -1) / people_ratings.sum(axis=1)
people_score.head()

In [ ]:
people_score.plot.hist(bins=200, figsize=(20, 10))

In [ ]:
#people_score.plot.hist(bins=200, figsize=(20, 10))
#sns.violinplot(x='type', y='score', data=people_score, inner='stick', bw=.3)
sns.violinplot(data=people_score.to_frame(), inner='stick', bw=.2)
plt.title('informovanost')
plt.xticks(rotation=15)

In [ ]:
#people_score.plot.hist(bins=200, )
#sns.violinplot(x='type', y='score', data=people_score, inner='stick', bw=.3)
data_to_plot = pd.merge(df_form[['Věk', 'Pohlaví']], people_score.to_frame(name='score'), left_index=True, right_index=True).sort_values(by='Věk')
plt.figure(figsize=(20, 10))
sns.violinplot(x='Věk', y='score', hue='Pohlaví', data=data_to_plot, inner='stick', bw=.3)
plt.title('průměrné hodnocení')
plt.xticks(rotation=15)

### analysis of open answers

In [ ]:
from urllib import request

data = request.urlopen('https://raw.githubusercontent.com/stopwords-iso/stopwords-cs/master/stopwords-cs.txt')
all_stopwords = [line.decode('utf-8').replace('\n', '') for line in data]
# todo: split them correctly and for vocalulary calculation use only emotes filtering
emotes_stopwords = [':d', ':', '\?', ';', ':-d', '\^\^', ':dd', 'xd', ':3', '=d', ':p',  ':/', ':>', '\+', 'o_o', 'd:', '>', ':\'d', ':"d', '\n', ':dd', ':ddd', ':dddd', 'x', 'xp']
my_stopwords = ['no', '', 'jo', 'sa', 'the', '=d', 'fakt', 'celkem', 'sem', 'ja', 'som', 'ako', 'of', 'nekdo', 'bych', 'sou', '2', 'jj', 'is', 'mam', '3', '\n']
all_stopwords += my_stopwords
all_stopwords += emotes_stopwords

def messages_to_words_hist(mess_series):
    mess_series = mess_series.fillna('')
    all_words = mess_series.str.split(' ', expand=True).stack().reset_index(drop=True, level=1)
    words_historgram = all_words.str.lower().str.strip('.,?-–()').value_counts()
    stopwords = list(set(all_stopwords).intersection(set(words_historgram.index)))
    words_historgram_cleaned = words_historgram.drop(stopwords)
    return words_historgram_cleaned


In [ ]:
best_mess = messages_to_words_hist(df_form['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'].str.contains('přednášky', regex=False)]['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'])

In [ ]:
best_mess = messages_to_words_hist(df_form['Co vás letos nejvíc zklamalo / nelíbilo se vám?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='Co vás letos nejvíc zklamalo / nelíbilo se vám?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['Co vás letos nejvíc zklamalo / nelíbilo se vám?'].str.contains('přednášky', regex=False)]['Co vás letos nejvíc zklamalo / nelíbilo se vám?'])

In [ ]:
best_mess = messages_to_words_hist(df_form['Co bychom podle vás měli do příštího roku zlepšit?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='Co bychom podle vás měli do příštího roku zlepšit?')
plt.xticks(rotation=30)

In [ ]:
best_mess = messages_to_words_hist(df_form['Co bychom podle vás měli do příštího roku zlepšit?'])
best_mess[:15].plot.bar(figsize=(30, 10), title='Co bychom podle vás měli do příštího roku zlepšit?')
plt.xticks(rotation=30)

In [ ]:
best_mess = messages_to_words_hist(df_form['S čím jste měli problémy?'])
best_mess[:15].plot.bar(figsize=(30, 10), title='S čím jste měli problémy?')
plt.xticks(rotation=30)

In [ ]:
best_mess = messages_to_words_hist(df_form['Uvítali byste nějaké další suvenýry?'])
best_mess[:15].plot.bar(figsize=(30, 10), title='Uvítali byste nějaké další suvenýry?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['Uvítali byste nějaké další suvenýry?'] != '']['Uvítali byste nějaké další suvenýry?'])

In [ ]:
best_mess = messages_to_words_hist(df_form['Hodnocení a poznámky ke grafice'])
best_mess[:15].plot.bar(figsize=(30, 10), title='Hodnocení a poznámky ke grafice')
plt.xticks(rotation=30)

In [ ]:
[i for i in open_answers if 'Komentář' not in i]

In [ ]:
best_mess = messages_to_words_hist(df_form['Jak jste byli spokojeni s moderátory soutěží a oficiálních pořadů?'])
best_mess[:15].plot.bar(figsize=(30, 10), title='Jak jste byli spokojeni s moderátory soutěží a oficiálních pořadů?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['Jak jste byli spokojeni s moderátory soutěží a oficiálních pořadů?'] != '']['Jak jste byli spokojeni s moderátory soutěží a oficiálních pořadů?'].tolist())

### analýza jednotlivých přednášek

In [ ]:
#column = 'Jak si plnit sny pomocí anime'
#column = 'Anime světy, ve kterých (ne)chcete žít'
#column = 'Nový kreslíř na scéně: umělá inteligence!'
#column = 'Sekiro, Nioh a ti další…'
#column = 'Pen & Paper role-playing hry'
#column = 'Rámen po česku'
#column = 'BTS Army sraz'
#column = 'K-pop – random dance CZHW'
#column = 'Historická přesnost zbrojí v anime na vybraných příkladech'
column = 'Letem jiným světem'

In [ ]:
not_interesting_values = [np.nan, 'Nezúčastnil(a) jsem se', 'Nedostal(a) jsem se']

df_filtered = df_form[~df_form[column + info_suffix].isin(not_interesting_values) | ~ df_form[column + fun_suffix].isin(not_interesting_values)]
# Create dimensions
age_dim = go.parcats.Dimension(
    values=df_filtered['Věk'],
    categoryorder='category ascending', label='Věk'
)

gender_dim = go.parcats.Dimension(
    values=df_filtered['Pohlaví'],
    categoryorder='category ascending', label='Pohlaví'
)

place_dim = go.parcats.Dimension(
    values=df_filtered['Bydliště'],
    categoryorder='category ascending', label='Bydliště'
)

num_visits_dim = go.parcats.Dimension(
    values=df_filtered['Kromě letošního ročníku jsem na Animefestu byl(a)'],
    categoryorder='category ascending', label='Kolikrát předtím na AF'
)

info_dim = go.parcats.Dimension(
    values=df_filtered[column + info_suffix],
    categoryorder='category ascending', label='Informativnost programu'
)

fun_dim = go.parcats.Dimension(
    values=df_filtered[column + fun_suffix],
    categoryorder='category ascending', label='Zábavnost programu'
)

# Create parcats trace
color = df_filtered['Pohlaví'].replace({'muž': 0, 'žena': 0.3, 'jiné': 0.6, np.nan: 1})
colorscale = [[0, 'steelblue'], [0.3, 'indianred'], [0.6, 'gray'], [1, 'dimgray']];

fig = go.Figure(data = [go.Parcats(dimensions=[age_dim, num_visits_dim, gender_dim, info_dim, fun_dim],
        line={'color': color, 'colorscale': colorscale},
        hoveron='color', hoverinfo='count+probability',
        labelfont={'size': 18, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')])

fig.show()
# todo: zkorelovat průměrnou spokojenost lidí s počtem AF návštěv

In [ ]:
program_info_score.loc[column]

In [ ]:
program_fun_score.loc[column]

In [ ]:
df_form[df_form['Historická přesnost zbrojí v anime na vybraných příkladech [Dozvěděl(a) jsem se zajímavé informace]'] == 'špatné'].to_dict()

In [ ]:
df_filtered[df_filtered['Komentář: '+column] != '']['Komentář: '+column].to_list()

In [ ]:
# todo: udělat violin plot průměrných hodnocení pro jednotlivé lidi a různé demografie. A kouknout na množství překryvů mezi hodnoceným programem časově, zda to šlo stihnout

In [ ]:
df_filtered[df_filtered['Komentář: '+column] != ''][['Komentář: '+column, column + info_suffix, column + fun_suffix, 'Pohlaví', 'Věk']]

### loading data from api

In [ ]:
schedule_entries = requests.get('https://www.animefest.cz/api/v1/schedule/entries/2/cs').json()
df_schedule = pd.DataFrame(schedule_entries['entries'])
df_schedule = df_schedule.set_index('scheduleEntryId')

In [ ]:
df_schedule.head()

In [ ]:
timetable_entries = requests.get('https://www.animefest.cz/api/v1/schedule/timetable/2/cs').json()
df_timetable = pd.DataFrame(timetable_entries['timetable'])
df_timetable = df_timetable.set_index('scheduleShowingId')
df_timetable['from'] = pd.to_datetime(df_timetable['from'])
df_timetable['to'] = pd.to_datetime(df_timetable['to'])

In [ ]:
df_timetable.head()

In [ ]:
locs_entries = requests.get('https://www.animefest.cz/api/v1/schedule/locations/2/cs').json()
df_locs = pd.DataFrame(locs_entries['locations'])
df_locs = df_locs.set_index('locationId')
df_locs['capacity'] = 0

In [ ]:
df_locs.loc[df_locs['name'] == 'Rotunda', 'capacity'] = 500
df_locs.loc[df_locs['name'] == 'A2, hlavní pódium', 'capacity'] = 450
df_locs.loc[df_locs['name'] == 'A2, studio stage', 'capacity'] = 330
df_locs.loc[df_locs['name'] == 'Sál Morava', 'capacity'] = 280
df_locs.loc[df_locs['name'] == 'Sál A (KC)', 'capacity'] = 300
df_locs.loc[df_locs['name'] == 'Sál B (KC)', 'capacity'] = 170
df_locs.loc[df_locs['name'] == 'A1, Stan bojovníků', 'capacity'] = 210  # papírově, ale nerozeznáme workshop od ukázky
df_locs.loc[df_locs['name'] == 'A1, Módní koutek', 'capacity'] = 30
df_locs.loc[df_locs['name'] == 'A1, Digital art', 'capacity'] = 20
df_locs.loc[df_locs['name'] == 'A1, stream', 'capacity'] = 608
df_locs.loc[df_locs['name'] == 'Mimo sály', 'capacity'] = 50  # asi, čeká se na upřesnění
df_locs.loc[df_locs['name'] == 'A2, workshopy', 'capacity'] = 10


In [ ]:
df_locs

##### fixing non-aligned names

In [ ]:
names_to_replace = {
    'BJD od Ludvíka XIV. až po BTS ': 'BJD od Ludvíka XIV. až po BTS', 
    'Gejša: Tajemství za stěnami čajovny ': 'Gejša: Tajemství za stěnami čajovny', 
    'Jak sehnat (nejen) jaoi v totalitním státu ': 'Jak sehnat (nejen) jaoi v totalitním státu',
    'Líčení pro trdla od trdla ': 'Líčení pro trdla od trdla', 'World Cosplay Panel': 'World cosplay panel',
    'Výroba vlastního diáře/skicáku ': 'Výroba vlastního diáře/skicáku', 
    'I pád je posun vpřed – workshop': 'I pád je posun vpřed', 'Sebeobrana po japonsku – workshop': 'Sebeobrana po japonsku',
    'Kendo – vystoupení/workshop': 'Kendo', 'Aikido Ikigai Dojo Brno – vystoupení/workshop': 'Aikido Ikigai Dojo Brno',
    'Aikikai Aikido Brno – vystoupení/workshop': 'Aikikai Aikido Brno', 'Yoshinkan aikido – vystoupení/workshop': 'Yoshinkan aikido',
    'Kensei Dojo Brno - vystoupení/workshop': 'Kensei Dojo Brno', 'Turnaj 1. kolo': 'Turnaj v šermu Aréna Alerie', 
    'Turnaj 2. kolo': 'Turnaj v šermu Aréna Alerie', 'Turnaj 3. kolo': 'Turnaj v šermu Aréna Alerie', 'Turnaj Finále': 'Turnaj v šermu Aréna Alerie',
}

for name_from, name_to in names_to_replace.items():
    df_schedule.loc[df_schedule['name'] == name_from, 'name'] = name_to


In [ ]:
for entry_name in df_schedule['name']:
    if not (entry_name + fun_suffix in df_form and entry_name + info_suffix in df_form):
        print(entry_name)
    #assert entry['name'] + fun_suffix in df_form and entry['name'] + info_suffix in df_form, entry['name']

In [ ]:
for column in program_columns:
    if not column in df_schedule['name'].tolist():
        print(column)
    assert column + fun_suffix in df_form and column + info_suffix in df_form, column

In [ ]:
df_timetable = df_timetable.drop(columns=['animefestYearId', 'language'])
df_locs = df_locs.drop(columns=['animefestYearId', 'language']).rename(columns={'name': 'location_name'})
df_timetable = df_timetable.join(df_schedule, on='scheduleEntryId').join(df_locs, on='locationId')
df_timetable.head()

In [ ]:
df_timetable[df_timetable['name'].isin(program_columns)].head()

In [ ]:
df_timetable[df_timetable['name'].isin(program_columns)].shape, len(df_timetable[df_timetable['name'].isin(program_columns)]['scheduleEntryId'].unique())

In [ ]:
df_timetable[df_timetable['name'].isin(program_columns) & (df_timetable['location_name'] == 'A1, Stan bojovníků')].head()

In [ ]:
df_timetable[df_timetable['name'].isin(program_columns)]['location_name'].unique()

In [ ]:
df_timetable[df_timetable['name'].isin(program_columns)].sort_values(by='scheduleEntryId').head()

##### seeing which program is several times in the schedule

In [ ]:
df_timetable[df_timetable['scheduleEntryId'].isin(df_timetable['scheduleEntryId'].value_counts()[df_timetable['scheduleEntryId'].value_counts() > 1].index)].head()

### filtering, keeping only entries in feedback form

In [ ]:
df_timetable = df_timetable[(df_timetable['name'] + info_suffix).isin(df_form.columns)]  # there is consistency check for both info and fun suffixes, using any of them arbitratily

In [ ]:
df_timetable[df_timetable['scheduleEntryId'].isin(df_timetable['scheduleEntryId'].value_counts()[df_timetable['scheduleEntryId'].value_counts() > 1].index)]

In [ ]:
pd.Series(df_timetable[df_timetable['scheduleEntryId'].isin(df_timetable['scheduleEntryId'].value_counts()[df_timetable['scheduleEntryId'].value_counts() > 1].index)]['name'].unique())
